In [ ]:
import pathlib
import numpy

from bluepyefe.extract import read_recordings, extract_efeatures_at_targets, compute_rheobase,\
    mean_efeatures, create_feature_protocol_files
from bluepyefe.plotting import plot_all_recordings_efeatures

from reader import wcp_reader
from targets import targets, ecode_to_index, ecodes_timings

In [ ]:
def build_metadata(cell_id="cell1", ephys_dir="./exp_patch_data/", repetition_as_different_cells=True):
    
    files_metadata  = {}
    
    if not(repetition_as_different_cells):
        files_metadata[cell_id] = {}
        
    for repetition in range(1, 5):
        
        if repetition_as_different_cells:
            current_id = f"{cell_id}_rep{repetition}"
            files_metadata[current_id] = {}
        else:
            current_id = cell_id
            
        for ecode in ecode_to_index:

            file_path = pathlib.Path(ephys_dir) / f"{cell_id}_run{repetition}.{ecode_to_index[ecode]}.wcp"
            
            if not file_path.is_file():
                print(f"Missing trace {file_path}")
                continue
            
            metadata= {
                "filepath": str(file_path),
                "i_unit": "pA",
                "t_unit": "s",
                "v_unit": "mV",
            }

            metadata.update(ecodes_timings[ecode])

            if ecode not in files_metadata[current_id]:
                files_metadata[current_id][ecode] = [metadata]
            else:
                files_metadata[current_id][ecode].append(metadata)

    return files_metadata

In [ ]:
output_directory = "./efeatures/"

cells = read_recordings(
    files_metadata=build_metadata(),
    recording_reader=wcp_reader
)

In [ ]:
extract_efeatures_at_targets(
    cells, 
    targets,
    threshold=-20.
)

In [ ]:
compute_rheobase(
    cells, 
    protocols_rheobase=['IDthres']
)

In [ ]:
protocols = mean_efeatures(cells, targets, use_global_rheobase=True)

In [ ]:
efeatures, protocol_definitions, current = create_feature_protocol_files(
    cells,
    protocols,
    output_directory=output_directory,
    threshold_nvalue_save=1,
    write_files=True,
)

In [ ]:
plot_all_recordings_efeatures(
    cells, protocols, output_dir=output_directory
)